# Stacked Regression on the Kaggle Housing Data

Richard Corrado richcorrado.github.io

The data files are stored in the same github directory as this notebook.  For the purposes of this exercise, we won't go into too many details of the datasets.  They were all generated from the same messy data set, but after EDA and feature engineering in R, choice and defintion of new features evolved. However many tidy data files were generated as saved during the learning process.  They have many features in common, but have important differences.

This notebook takes 8 datasets with 8 different regression models and applies Breiman's stacked regression (Breiman, L. Mach Learn (1996) 24: 49. doi:10.1007/BF00117832, available at statistics.berkeley.edu/sites/default/files/tech-reports/367.pdf).  The hyperparameters of the regression models were tuned to the individual datasets via 10-fold CV.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.precision',5)
from scipy import stats
from scipy import optimize

from sklearn import linear_model, svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, cross_val_predict, KFold, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.pipeline import Pipeline
import xgboost as xgb
from xgboost.training import train
from xgboost.sklearn import XGBClassifier

import matplotlib
# this is needed for interactive plots to be displayed properly
matplotlib.use('Agg')
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib import pyplot
rcParams['figure.figsize'] = 12, 4
# allow interactive plots
%matplotlib notebook

In [2]:
# def to compare goodness of fit on training set
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [3]:
# these files have all features
train_1_9_df = pd.read_csv("train-1-9.csv")
test_1_9_df = pd.read_csv("test-1-9.csv")

# these files have all features
train_1_7_df = pd.read_csv("train-1-7.csv")
test_1_7_df = pd.read_csv("test-1-7.csv")

# these files have all features
train_1_3_df = pd.read_csv("train-1-3.csv")
test_1_3_df = pd.read_csv("test-1-3.csv")

train_1_3_dropped_df = pd.read_csv("train-1-3-dropped.csv")
test_1_3_dropped_df = pd.read_csv("test-1-3-dropped.csv")

train_1_2_df = pd.read_csv("train-1-2.csv")
test_1_2_df = pd.read_csv("test-1-2.csv")

# Legacy files with all features as of 12/31/16
train_12_31_df = pd.read_csv("train-12-31.csv")
test_12_31_df = pd.read_csv("test-12-31.csv")

# these have Amit Choudhary's features (there is code below to deal with some differences)
label_AC_df = pd.read_csv("AC_label.csv")
train_AC_df = pd.read_csv("AC_train.csv")
test_AC_df = pd.read_csv("AC_test.csv")

# low variance columns have been dropped in hyperparameter-search.py
# these files have dropped cat features with less than 10 nonzero entries
train_12_31_dropped_df = pd.read_csv("train-12-31-dropped.csv")
test_12_31_dropped_df = pd.read_csv("test-12-31-dropped.csv")

In [4]:
# Set up predictors and response
y_train_1_9 = train_1_9_df['LogSalePrice'].values
x_train_1_9 = train_1_9_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_1_9 = test_1_9_df.drop(['Id'],axis=1).values

y_train_1_7 = train_1_7_df['LogSalePrice'].values
x_train_1_7 = train_1_7_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_1_7 = test_1_7_df.drop(['Id'],axis=1).values

y_train_1_3 = train_1_3_df['LogSalePrice'].values
x_train_1_3 = train_1_3_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_1_3 = test_1_3_df.drop(['Id'],axis=1).values

y_train_1_3_dropped = train_1_3_dropped_df['LogSalePrice'].values
x_train_1_3_dropped = train_1_3_dropped_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_1_3_dropped = test_1_3_dropped_df.drop(['Id'],axis=1).values

y_train_1_2 = train_1_2_df['LogSalePrice'].values
x_train_1_2 = train_1_2_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_1_2 = test_1_2_df.drop(['Id'],axis=1).values


y_train_12_31 = train_12_31_df['LogSalePrice'].values
x_train_12_31 = train_12_31_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_12_31 = test_12_31_df.drop(['Id'],axis=1).values

y_train_12_31_dropped = train_12_31_dropped_df['LogSalePrice'].values
x_train_12_31_dropped = train_12_31_dropped_df.drop(['Id','LogSalePrice'],axis=1).values
x_test_12_31_dropped = test_12_31_dropped_df.drop(['Id'],axis=1).values

# AC's features are a bit different
y_train_AC = label_AC_df['SalePrice'].values
x_train_AC = train_AC_df.drop(['Id','_RoofMatl_ClyTile'],axis=1).values
x_test_AC = test_AC_df.drop(['Id'],axis=1).values

In [5]:
print("1-9 features training set size:", x_train_1_9.shape)
print("1-9 features test set size:", x_test_1_9.shape)

print("1-7 features training set size:", x_train_1_7.shape)
print("1-7 features test set size:", x_test_1_7.shape)

print("1-3 features training set size:", x_train_1_3.shape)
print("1-3 features test set size:", x_test_1_3.shape)

print("1-3 w dropfeatures training set size:", x_train_1_3_dropped.shape)
print("1-3 w drop features test set size:", x_test_1_3_dropped.shape)

print("1-2 features training set size:", x_train_1_2.shape)
print("1-2 features test set size:", x_test_1_2.shape)

print("12-31 features training set size:", x_train_12_31.shape)
print("12-31 features test set size:", x_test_12_31.shape)
print("12-31 w drop features training set size:", x_train_12_31_dropped.shape)
print("12-31 w drop features test set size:", x_test_12_31_dropped.shape)

print("AC features training set size:", x_train_AC.shape)
print("AC features training response set size:", y_train_AC.shape)
print("AC features test set size:", x_test_AC.shape)

('1-9 features training set size:', (1460, 391))
('1-9 features test set size:', (1459, 391))
('1-7 features training set size:', (1460, 403))
('1-7 features test set size:', (1459, 403))
('1-3 features training set size:', (1460, 384))
('1-3 features test set size:', (1459, 384))
('1-3 w dropfeatures training set size:', (1460, 92))
('1-3 w drop features test set size:', (1459, 92))
('1-2 features training set size:', (1460, 437))
('1-2 features test set size:', (1459, 437))
('12-31 features training set size:', (1460, 425))
('12-31 features test set size:', (1459, 425))
('12-31 w drop features training set size:', (1460, 366))
('12-31 w drop features test set size:', (1459, 366))
('AC features training set size:', (1460, 406))
('AC features training response set size:', (1460,))
('AC features test set size:', (1459, 406))


Design and response.  We only need one response vector:

In [6]:
y_train = y_train_1_9
x_train = [x_train_1_9, x_train_1_7, x_train_1_3, x_train_1_3_dropped, x_train_1_2,
           x_train_12_31, x_train_12_31_dropped, x_train_AC]
x_test = [x_test_1_9, x_test_1_7, x_test_1_3, x_test_1_3_dropped, x_test_1_2,
          x_test_12_31, x_test_12_31_dropped, x_test_AC]

We'll use KFold to generate our folds.

In [8]:
kfold = KFold(n_splits=10, random_state=7)

Models and hyper parameters:

In [10]:
rms_1_9 = [linear_model.Ridge(alpha = 24),
           linear_model.Lasso(alpha=0.0004, max_iter=50000),
           linear_model.LassoLars(alpha=0.00012, max_iter=50000),
           linear_model.ElasticNet(alpha = 0.0009, l1_ratio=0.54, max_iter=15000, random_state=7),
           RandomForestRegressor(n_estimators = 1500, max_depth = 16, random_state = 7),
           svm.SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=0.036, gamma=0.0009, kernel='rbf',
                   max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           xgb.XGBRegressor(max_depth=2, min_child_weight=1.1, gamma=0, subsample=1, colsample_bytree=0.8,
                            reg_alpha=0.1, reg_lambda=0.2, learning_rate=0.06, n_estimators=900, seed=42,
                            nthread=-1, silent=1),
           linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=33)
           ]

rms_1_7 = [linear_model.Ridge(alpha = 14),
          linear_model.Lasso(alpha=0.0005, max_iter=50000),
          linear_model.LassoLars(alpha=0.00011, max_iter=50000),
          linear_model.ElasticNet(alpha = 0.0009, l1_ratio=0.54, max_iter=15000, random_state=7),
          RandomForestRegressor(n_estimators = 600, max_depth = 21, random_state = 7),
          svm.SVR(C=25.2, cache_size=200, coef0=0.0, degree=3, epsilon=0.0037, gamma=0.00006, kernel='rbf',
                  max_iter=-1, shrinking=True, tol=0.001, verbose=False),
          xgb.XGBRegressor(max_depth = 3, min_child_weight = 3, gamma = 0, subsample = 0.92, colsample_bytree = 0.6,
                           reg_alpha = 0.2, reg_lambda = 0.2, learning_rate = 0.1, n_estimators = 1100, seed = 42,
                           nthread = -1, silent = 1),
          linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=28)
          ]

rms_1_3 = [linear_model.Ridge(alpha = 26),
           linear_model.Lasso(alpha=0.00068, max_iter=50000),
           linear_model.LassoLars(alpha=0.00012, max_iter=50000),
           linear_model.ElasticNet(alpha = 0.0014, l1_ratio=0.49, max_iter=15000, random_state=7),
           RandomForestRegressor(n_estimators = 2300, max_depth = 19, random_state = 7),
           svm.SVR(C=1.95, cache_size=200, coef0=0.0, degree=3, epsilon=0.0167, gamma='auto',kernel='rbf',
                   max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           xgb.XGBRegressor(max_depth = 2, min_child_weight = 3.1, gamma = 0.015, subsample = 0.81,
                            colsample_bytree = 0.80, reg_alpha = 0.19, reg_lambda = 0.2, learning_rate = 0.071,
                            n_estimators = 3400, seed = 42, nthread = -1, silent = 1),
           linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=37)
           ]

rms_1_3_dropped = [linear_model.Ridge(alpha = 2.6),
                   linear_model.Lasso(alpha=0.000151, max_iter=50000),
                   linear_model.LassoLars(alpha=0.00014, max_iter=50000),
                   linear_model.ElasticNet(alpha = 0.0002, l1_ratio=0.00015, max_iter=15000, random_state=7),
                   RandomForestRegressor(n_estimators = 600, max_depth = 19, random_state = 7),
                   svm.SVR(C=31.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.02535, gamma=0.000033,
                           kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
                   xgb.XGBRegressor( max_depth = 3, min_child_weight = 3.1, gamma = 0.018, subsample = 0.8,
                                     colsample_bytree = 0.8, reg_alpha = 0.2, reg_lambda = 0.2, learning_rate = 0.07,
                                     n_estimators = 900, seed = 42, nthread = -1, silent = 1),
                   linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=80)
                   ]

rms_1_2 = [linear_model.Ridge(alpha = 19.2),
           linear_model.Lasso(alpha=0.000838, max_iter=50000),
           linear_model.LassoLars(alpha=0.000181, max_iter=50000),
           linear_model.ElasticNet(alpha = 0.00142, l1_ratio=0.501, max_iter=15000, random_state=7),
           RandomForestRegressor(n_estimators = 620, max_depth = 15, random_state = 7),
           svm.SVR(C=1.56, cache_size=200, coef0=0.0, degree=3, epsilon=0.00735, gamma='auto',
                   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           xgb.XGBRegressor( max_depth = 2, min_child_weight = 3.1, gamma = 0.00968, subsample = 0.778,
                             colsample_bytree = 0.83, reg_alpha = 0.2085, reg_lambda = 0.1991,
                             learning_rate = 0.05, n_estimators = 2000, seed = 42, nthread = -1,
                             silent = 1),
           linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=28)
           ]

rms_12_31 = [linear_model.Ridge(alpha = 74.682),
             linear_model.Lasso(alpha=0.000866, max_iter=50000),
             linear_model.LassoLars(alpha=0.000194, max_iter=50000),
             linear_model.ElasticNet(alpha = 0.001735, l1_ratio=0.5, max_iter=15000, random_state=7),
             RandomForestRegressor(n_estimators=570, max_depth=26, random_state=7),
             svm.SVR(C=1.15, cache_size=200, coef0=0.0, degree=3, epsilon=0.0222, gamma='auto', kernel='rbf',
                     max_iter=-1, shrinking=True, tol=0.001, verbose=False),
             xgb.XGBRegressor(max_depth = 3, min_child_weight = 1.1, gamma = 0.0529, subsample = 0.8,
                              colsample_bytree = 0.202, reg_alpha = 0.152, reg_lambda = 0.19705,
                              learning_rate = 0.2, n_estimators = 13000, seed = 42, nthread = -1, silent = 1),
             linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=27)
             ]


rms_12_31_dropped = [linear_model.Ridge(alpha = 78.17),
                     linear_model.Lasso(alpha=0.0007753, max_iter=50000),
                     linear_model.LassoLars(alpha=0.000122, max_iter=50000),
                     linear_model.ElasticNet(alpha = 0.001468, l1_ratio=0.5, max_iter=15000, random_state=7),
                     RandomForestRegressor(n_estimators=620, max_depth=15, random_state=7),
                     svm.SVR(C=1.2, cache_size=200, coef0=0.0, degree=3, epsilon=0.0211, gamma='auto', kernel='rbf',
                             max_iter=-1, shrinking=True, tol=0.001, verbose=False),
                     xgb.XGBRegressor(max_depth = 3, min_child_weight = 2.02,gamma = 0, subsample = 0.81985,
                                      colsample_bytree = 0.9, reg_alpha = 0.193, reg_lambda = 0.200481,
                                      learning_rate = 0.0141, n_estimators = 3500, seed = 42, nthread = -1,
                                      silent = 1),
                     linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=36)
                     ]

rms_AC = [linear_model.Ridge(alpha = 58.28),
          linear_model.Lasso(alpha=0.000378, max_iter=50000),
          linear_model.LassoLars(alpha=0.0005971, max_iter=50000),
          linear_model.ElasticNet(alpha = 0.000739, l1_ratio=00.5017, max_iter=15000, random_state=7),
          RandomForestRegressor(n_estimators=780, max_depth=22, random_state=7),
          svm.SVR(C=0.82, cache_size=200, coef0=0.0, degree=3, epsilon=0.00296, gamma='auto', kernel='rbf',
                  max_iter=-1, shrinking=True, tol=0.001, verbose=False),
          xgb.XGBRegressor(max_depth = 1, min_child_weight = 1, gamma = 0.02969, subsample = 0.9017,
                           colsample_bytree = 0.8, reg_alpha = 0.2902, reg_lambda = 0.1993, learning_rate = 0.21,
                           n_estimators = 22000, seed = 42, nthread = -1, silent = 1),
          linear_model.OrthogonalMatchingPursuit(n_nonzero_coefs=79)
          ]


rms = [rms_1_9, rms_1_7, rms_1_3, rms_1_3_dropped, rms_1_2, rms_12_31, rms_12_31_dropped, rms_AC]

## Stacked Regression in a Nutshell

In stacked regression, we:

1. Use KFold to split the training set $S$ into $\otimes_i S_i$ and the same with the test set $T$.

2. We fit the models on the "in fold" sets 
$$S_{-j} = \otimes_{i \neq j} S_j.$$

3. We use the fitted models to predict the response on $S_j$.

4. Iterating this procedure over the folds allows us to assemble the response on the whole of 
$$ \otimes_j S_j = S.$$
These response predictions are the training metafeatures.

5. We use the training metafeatures to train a second stage regressor.

6. Now use the stage one models to assemble the test metafeatures on $T$.

7. Use the stage 2 regressor to predict the test response from the test metafeatures.

Now define some constants:

In [11]:
n_data = 8
n_models = 8

n_train_obs = x_train[0].shape[0]
n_test_obs = x_test[0].shape[0]

n_feat = []
for i in range(0, n_data):
    n_feat.append(x_train[i].shape[1])

Note that the number of observations and number of features can differ between the datasets, but the definitions above will keep track of this.  Now define a placeholder for the metafeatures:

In [12]:
z_train = np.zeros((n_train_obs, n_data, n_models))
z_test = np.zeros((n_train_obs, n_data, n_models))

Next we build the training metafeatures:

In [13]:
for i in range(0, n_data):
    print "Dataset: ", i
    # implementing the 10-fold CV sets
    for fold, (train_idx, test_idx) in enumerate(kfold.split(x_train[i])):
        print "Fold: ", fold
        # in and out of fold parts of design matrix
        x_if, x_oof = x_train[i][train_idx], x_train[i][test_idx]
        # we don't need the response for the out of fold data
        y_if = y_train[train_idx]
        # this is the most efficient loop to do fit and predict
        for a in range(0, n_models):
            print "Fitting model", a+1, "on fold ", fold+1, "of dataset ", i+1
            # fit on in fold data
            rms[i][a].fit(x_if,y_if)
            # metafeatures are the predictions on the out of fold data
            z_train[test_idx,i,a] = rms[i][a].predict(x_oof)

Dataset:  0
Fold:  0
Fitting model 1 on fold  1 of dataset  1
Fitting model 2 on fold  1 of dataset  1
Fitting model 3 on fold  1 of dataset  1
Fitting model 4 on fold  1 of dataset  1
Fitting model 5 on fold  1 of dataset  1
Fitting model 6 on fold  1 of dataset  1
Fitting model 7 on fold  1 of dataset  1
Fitting model 8 on fold  1 of dataset  1
Fold:  1
Fitting model 1 on fold  2 of dataset  1
Fitting model 2 on fold  2 of dataset  1
Fitting model 3 on fold  2 of dataset  1
Fitting model 4 on fold  2 of dataset  1
Fitting model 5 on fold  2 of dataset  1
Fitting model 6 on fold  2 of dataset  1
Fitting model 7 on fold  2 of dataset  1
Fitting model 8 on fold  2 of dataset  1
Fold:  2
Fitting model 1 on fold  3 of dataset  1
Fitting model 2 on fold  3 of dataset  1
Fitting model 3 on fold  3 of dataset  1
Fitting model 4 on fold  3 of dataset  1
Fitting model 5 on fold  3 of dataset  1
Fitting model 6 on fold  3 of dataset  1
Fitting model 7 on fold  3 of dataset  1
Fitting model 8 on

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 11 iterations, i.e. alpha=1.216e-03, with an active set of 11 regressors, and the smallest cholesky pivot element being 2.107e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=9.560e-04, with an active set of 12 regressors, and the smallest cholesky pivot element being 8.025e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 18 iterations, i.e. alpha=8.699e-04, with an active set of 18 regressors, and the smallest cholesky pivot element being 2.107e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least

Fitting model 5 on fold  1 of dataset  2
Fitting model 6 on fold  1 of dataset  2
Fitting model 7 on fold  1 of dataset  2
Fitting model 8 on fold  1 of dataset  2
Fold:  1
Fitting model 1 on fold  2 of dataset  2
Fitting model 2 on fold  2 of dataset  2
Fitting model 3 on fold  2 of dataset  2
Fitting model 4 on fold  2 of dataset  2


/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=1.065e-03, with an active set of 14 regressors, and the smallest cholesky pivot element being 4.712e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=1.065e-03, with an active set of 14 regressors, and the smallest cholesky pivot element being 8.429e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 32 iterations, i.e. alpha=5.275e-04, with an active set of 30 regressors, and the smallest cholesky pivot element being 4.712e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least

Fitting model 5 on fold  2 of dataset  2
Fitting model 6 on fold  2 of dataset  2
Fitting model 7 on fold  2 of dataset  2
Fitting model 8 on fold  2 of dataset  2
Fold:  2
Fitting model 1 on fold  3 of dataset  2
Fitting model 2 on fold  3 of dataset  2
Fitting model 3 on fold  3 of dataset  2
Fitting model 4 on fold  3 of dataset  2


...

Fitting model 5 on fold  9 of dataset  2
Fitting model 6 on fold  9 of dataset  2
Fitting model 7 on fold  9 of dataset  2
Fitting model 8 on fold  9 of dataset  2
Fold:  9
Fitting model 1 on fold  10 of dataset  2
Fitting model 2 on fold  10 of dataset  2
Fitting model 3 on fold  10 of dataset  2
Fitting model 4 on fold  10 of dataset  2


/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 111 iterations, i.e. alpha=1.207e-04, with an active set of 101 regressors, and the smallest cholesky pivot element being 7.451e-08
  ConvergenceWarning)


Fitting model 5 on fold  10 of dataset  2
Fitting model 6 on fold  10 of dataset  2
Fitting model 7 on fold  10 of dataset  2
Fitting model 8 on fold  10 of dataset  2
Dataset:  2
Fold:  0
Fitting model 1 on fold  1 of dataset  3
Fitting model 2 on fold  1 of dataset  3
Fitting model 3 on fold  1 of dataset  3
Fitting model 4 on fold  1 of dataset  3
Fitting model 5 on fold  1 of dataset  3
Fitting model 6 on fold  1 of dataset  3
Fitting model 7 on fold  1 of dataset  3
Fitting model 8 on fold  1 of dataset  3
Fold:  1
Fitting model 1 on fold  2 of dataset  3
Fitting model 2 on fold  2 of dataset  3
Fitting model 3 on fold  2 of dataset  3
Fitting model 4 on fold  2 of dataset  3
Fitting model 5 on fold  2 of dataset  3
Fitting model 6 on fold  2 of dataset  3
Fitting model 7 on fold  2 of dataset  3
Fitting model 8 on fold  2 of dataset  3
Fold:  2
Fitting model 1 on fold  3 of dataset  3
Fitting model 2 on fold  3 of dataset  3
Fitting model 3 on fold  3 of dataset  3
Fitting model 

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 20 iterations, i.e. alpha=8.727e-04, with an active set of 20 regressors, and the smallest cholesky pivot element being 2.581e-08
  ConvergenceWarning)


Fitting model 5 on fold  4 of dataset  8
Fitting model 6 on fold  4 of dataset  8
Fitting model 7 on fold  4 of dataset  8
Fitting model 8 on fold  4 of dataset  8
Fold:  4
Fitting model 1 on fold  5 of dataset  8
Fitting model 2 on fold  5 of dataset  8
Fitting model 3 on fold  5 of dataset  8
Fitting model 4 on fold  5 of dataset  8
Fitting model 5 on fold  5 of dataset  8
Fitting model 6 on fold  5 of dataset  8
Fitting model 7 on fold  5 of dataset  8
Fitting model 8 on fold  5 of dataset  8
Fold:  5
Fitting model 1 on fold  6 of dataset  8
Fitting model 2 on fold  6 of dataset  8
Fitting model 3 on fold  6 of dataset  8
Fitting model 4 on fold  6 of dataset  8
Fitting model 5 on fold  6 of dataset  8
Fitting model 6 on fold  6 of dataset  8
Fitting model 7 on fold  6 of dataset  8
Fitting model 8 on fold  6 of dataset  8
Fold:  6
Fitting model 1 on fold  7 of dataset  8
Fitting model 2 on fold  7 of dataset  8
Fitting model 3 on fold  7 of dataset  8
Fitting model 4 on fold  7 of 

Next, reshape the array of metafeatures:

In [14]:
print("Metafeature set size:", z_train.shape)
print("Response set size:", y_train.shape)

z_train_mat = np.reshape(z_train, (n_train_obs, n_data * n_models))

print("Metafeature set size:", z_train_mat.shape)

('Metafeature set size:', (1460, 8, 8))
('Response set size:', (1460,))
('Metafeature set size:', (1460, 64))


As a 2nd stage model, we will use Nonnegative Least Squares, as in the original Breiman paper. The necessary function is part of the scipy optimize library:

In [15]:
# fit intercept by adding column of ones
ones_mat = np.ones(n_train_obs)
Z_mat = np.insert(z_train_mat, 0, ones_mat, axis=1)

# optimize NNLS model
nnLS = optimize.nnls(Z_mat, y_train)

The resulting model is quite sparse in this case:

In [19]:
nnLS[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.0159191 ,
        0.02629472,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.37954146,  0.34262723,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.19194826,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.04351285,  0.        ,  0.        ])

Next we build the test metafeatures:

In [20]:
z_test = np.zeros((n_test_obs, n_data, n_models))

for i in range(0, n_data):
    print "Dataset: ", i
    for a in range(0, n_models):
        print "Fitting model", a + 1, " on training dataset ", i + 1
        rms[i][a].fit(x_train[i], y_train)

        for fold, idx in enumerate(kfold.split(x_test[i])):
            print "Fold: ", fold
            train_idx, test_idx = idx
            x_oof = x_test[i][test_idx]
            print "Fitting model", a+1, "on fold ", fold+1, "of test dataset ", i+1
            z_test[test_idx,i,a] = rms[i][a].predict(x_oof)
            
z_test_mat = np.reshape(z_test, (n_test_obs, n_data * n_models))

Dataset:  0
Fitting model 1  on training dataset  1
Fold:  0
Fitting model 1 on fold  1 of test dataset  1
Fold:  1
Fitting model 1 on fold  2 of test dataset  1
Fold:  2
Fitting model 1 on fold  3 of test dataset  1
Fold:  3
Fitting model 1 on fold  4 of test dataset  1
Fold:  4
Fitting model 1 on fold  5 of test dataset  1
Fold:  5
Fitting model 1 on fold  6 of test dataset  1
Fold:  6
Fitting model 1 on fold  7 of test dataset  1
Fold:  7
Fitting model 1 on fold  8 of test dataset  1
Fold:  8
Fitting model 1 on fold  9 of test dataset  1
Fold:  9
Fitting model 1 on fold  10 of test dataset  1
Fitting model 2  on training dataset  1
Fold:  0
Fitting model 2 on fold  1 of test dataset  1
Fold:  1
Fitting model 2 on fold  2 of test dataset  1
Fold:  2
Fitting model 2 on fold  3 of test dataset  1
Fold:  3
Fitting model 2 on fold  4 of test dataset  1
Fold:  4
Fitting model 2 on fold  5 of test dataset  1
Fold:  5
Fitting model 2 on fold  6 of test dataset  1
Fold:  6
Fitting model 2 on

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 22 iterations, i.e. alpha=7.757e-04, with an active set of 20 regressors, and the smallest cholesky pivot element being 9.657e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=7.754e-04, with an active set of 21 regressors, and the smallest cholesky pivot element being 9.657e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=7.754e-04, with an active set of 21 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least

Fold:  0
Fitting model 4 on fold  1 of test dataset  2
Fold:  1
Fitting model 4 on fold  2 of test dataset  2
Fold:  2
Fitting model 4 on fold  3 of test dataset  2
Fold:  3
Fitting model 4 on fold  4 of test dataset  2
Fold:  4
Fitting model 4 on fold  5 of test dataset  2
Fold:  5
Fitting model 4 on fold  6 of test dataset  2
Fold:  6
Fitting model 4 on fold  7 of test dataset  2
Fold:  7
Fitting model 4 on fold  8 of test dataset  2
Fold:  8
Fitting model 4 on fold  9 of test dataset  2
Fold:  9
Fitting model 4 on fold  10 of test dataset  2
Fitting model 5  on training dataset  2
Fold:  0
Fitting model 5 on fold  1 of test dataset  2
Fold:  1
Fitting model 5 on fold  2 of test dataset  2
Fold:  2
Fitting model 5 on fold  3 of test dataset  2
Fold:  3
Fitting model 5 on fold  4 of test dataset  2
Fold:  4
Fitting model 5 on fold  5 of test dataset  2
Fold:  5
Fitting model 5 on fold  6 of test dataset  2
Fold:  6
Fitting model 5 on fold  7 of test dataset  2
Fold:  7
Fitting model 5

Apply the NNLS solution to obtain the stage 2 prediction:

In [21]:
ones_mat = np.ones(n_test_obs)
Z_test_mat = np.insert(z_test_mat, 0, ones_mat, axis=1)

y_stacking_pred = np.dot(Z_test_mat,nnLS[0])
y_stacking_pred = np.exp(y_stacking_pred) # response of model was log of SalePrice

We can write the predictions to a file, as for kaggle submission:

In [23]:
pred_stacking_df = pd.DataFrame(y_stacking_pred, index=test_1_9_df["Id"], columns=["SalePrice"])
pred_stacking_df.to_csv('stacking_output.csv', header=True, index_label='Id')